In [ ]:
from google.cloud import bigquery
import os

In [ ]:
## Project Metadata

PROJECT_ID = "qwiklabs-gcp-00-d269cced691e"
DATASET_ID = "fraud_detection"
RAW_TABLE = "fraud_data_raw"
TRAINING_TABLE = "fraud_training_data"

GCS_URI = "gs://labs.roitraining.com/data-to-ai-workshop/fraud_data_raw.csv"


In [ ]:
## Connect BigQuery Client and Create Dataset

client = bigquery.Client(project=PROJECT_ID)

dataset_ref = bigquery.Dataset(f"{PROJECT_ID}.{DATASET_ID}")
dataset_ref.location = "US"

try:
    client.create_dataset(dataset_ref)
    print("Dataset created.")
except Exception:
    print("Dataset already exists.")


Dataset created.


In [ ]:
## Specify the Table name and Ingest the data
table_id = f"{PROJECT_ID}.{DATASET_ID}.{RAW_TABLE}"

job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
)

load_job = client.load_table_from_uri(
    GCS_URI,
    table_id,
    job_config=job_config,
)

load_job.result()
print("Raw data loaded into BigQuery.")


Raw data loaded into BigQuery.


In [ ]:
## Feature Engineering Using Pandas
## Load data from BigQuery into Pandas dataframe

import pandas as pd

query = f"""
SELECT *
FROM `{PROJECT_ID}.{DATASET_ID}.{RAW_TABLE}`
"""

df = client.query(query).to_dataframe()
df.head()

,Applicant_ID,Age,Employment_Status,Income,Number_of_Dependents,Amount_Requested,Previous_Assistance_Received,Previous_Assistance_Date,Supporting_Doc_Verified,Application_Frequency_Last_Year,IP_Address,Device_Type,Application_Date,Fraudulent
0,217,65,Unemployed,28984,4,5872,False,NaT,False,1,156.133.45.45,Mobile,2024-08-18,0
1,226,54,Self-Employed,0,1,6631,False,NaT,False,1,245.13.80.245,Tablet,2024-05-11,0
2,240,26,Self-Employed,64477,5,8612,False,NaT,True,1,213.103.170.95,Mobile,2024-08-14,0
3,252,28,Unemployed,28576,4,2951,False,NaT,True,1,234.179.149.207,Desktop,2024-06-12,0
4,266,43,Employed,44930,5,2324,False,NaT,False,1,66.109.96.227,Mobile,2024-08-16,0


In [ ]:
## Transformation Task 1: One-hot encode the Employment_Status and Device_Type fields

df = pd.get_dummies(
    df,
    columns=["Employment_Status", "Device_Type"],
    prefix=["Employment_Status", "Device_Type"]
)
df.head(10)

,Applicant_ID,Age,Income,Number_of_Dependents,Amount_Requested,Previous_Assistance_Received,Previous_Assistance_Date,Supporting_Doc_Verified,Application_Frequency_Last_Year,IP_Address,Application_Date,Fraudulent,Employment_Status_Employed,Employment_Status_Self-Employed,Employment_Status_Unemployed,Device_Type_Desktop,Device_Type_Mobile,Device_Type_Tablet
0,217,65,28984,4,5872,False,NaT,False,1,156.133.45.45,2024-08-18,0,False,False,True,False,True,False
1,226,54,0,1,6631,False,NaT,False,1,245.13.80.245,2024-05-11,0,False,True,False,False,False,True
2,240,26,64477,5,8612,False,NaT,True,1,213.103.170.95,2024-08-14,0,False,True,False,False,True,False
3,252,28,28576,4,2951,False,NaT,True,1,234.179.149.207,2024-06-12,0,False,False,True,True,False,False
4,266,43,44930,5,2324,False,NaT,False,1,66.109.96.227,2024-08-16,0,True,False,False,False,True,False
5,281,32,25895,3,5411,False,NaT,False,1,202.173.84.210,2024-04-30,1,False,False,True,True,False,False
6,300,30,0,4,2245,False,NaT,True,1,178.95.218.82,2024-05-15,0,False,False,True,False,False,True
7,303,25,36223,5,2409,False,NaT,True,1,202.173.163.25,2024-03-24,0,False,True,False,False,False,True
8,330,35,42432,2,793,False,NaT,True,1,114.135.28.171,2024-01-14,0,True,False,False,False,False,True
9,353,46,0,3,8430,False,NaT,True,1,191.195.123.68,2024-07-24,0,False,True,False,True,False,False


In [ ]:
## Transformation Task 2: Break the age field into bins, and one-hot encode the bins

age_bins_cutoff = [18, 25, 35, 45, 55, 120]
age_labels = ["18_24", "25_34", "35_44", "45_54", "55_plus"]

df["Age_Bins"] = pd.cut(df["Age"], bins=age_bins_cutoff, labels=age_labels, right=False)

df = pd.get_dummies(df, columns=["Age_Bins"], prefix="Age")
df.head(10)

,Applicant_ID,Age,Income,Number_of_Dependents,Amount_Requested,Previous_Assistance_Received,Previous_Assistance_Date,Supporting_Doc_Verified,Application_Frequency_Last_Year,IP_Address,...,Employment_Status_Self-Employed,Employment_Status_Unemployed,Device_Type_Desktop,Device_Type_Mobile,Device_Type_Tablet,Age_18_24,Age_25_34,Age_35_44,Age_45_54,Age_55_plus
0,217,65,28984,4,5872,False,NaT,False,1,156.133.45.45,...,False,True,False,True,False,False,False,False,False,True
1,226,54,0,1,6631,False,NaT,False,1,245.13.80.245,...,True,False,False,False,True,False,False,False,True,False
2,240,26,64477,5,8612,False,NaT,True,1,213.103.170.95,...,True,False,False,True,False,False,True,False,False,False
3,252,28,28576,4,2951,False,NaT,True,1,234.179.149.207,...,False,True,True,False,False,False,True,False,False,False
4,266,43,44930,5,2324,False,NaT,False,1,66.109.96.227,...,False,False,False,True,False,False,False,True,False,False
5,281,32,25895,3,5411,False,NaT,False,1,202.173.84.210,...,False,True,True,False,False,False,True,False,False,False
6,300,30,0,4,2245,False,NaT,True,1,178.95.218.82,...,False,True,False,False,True,False,True,False,False,False
7,303,25,36223,5,2409,False,NaT,True,1,202.173.163.25,...,True,False,False,False,True,False,True,False,False,False
8,330,35,42432,2,793,False,NaT,True,1,114.135.28.171,...,False,False,False,False,True,False,False,True,False,False
9,353,46,0,3,8430,False,NaT,True,1,191.195.123.68,...,True,False,True,False,False,False,False,False,True,False


In [ ]:
## Transformation Task 3: Create a field called Income-to-Amount-Requested which is the ratio of those fields

df["Income_to_Amount_Requested"] = (
    df["Income"] / df["Amount_Requested"]
)

## Add Zero Handling?
df["Income_to_Amount_Requested"] = df["Income_to_Amount_Requested"].replace([float("inf")], 0)

df.head(10)

,Applicant_ID,Age,Income,Number_of_Dependents,Amount_Requested,Previous_Assistance_Received,Previous_Assistance_Date,Supporting_Doc_Verified,Application_Frequency_Last_Year,IP_Address,...,Employment_Status_Unemployed,Device_Type_Desktop,Device_Type_Mobile,Device_Type_Tablet,Age_18_24,Age_25_34,Age_35_44,Age_45_54,Age_55_plus,Income_to_Amount_Requested
0,217,65,28984,4,5872,False,NaT,False,1,156.133.45.45,...,True,False,True,False,False,False,False,False,True,4.935967
1,226,54,0,1,6631,False,NaT,False,1,245.13.80.245,...,False,False,False,True,False,False,False,True,False,0.0
2,240,26,64477,5,8612,False,NaT,True,1,213.103.170.95,...,False,False,True,False,False,True,False,False,False,7.486879
3,252,28,28576,4,2951,False,NaT,True,1,234.179.149.207,...,True,True,False,False,False,True,False,False,False,9.683497
4,266,43,44930,5,2324,False,NaT,False,1,66.109.96.227,...,False,False,True,False,False,False,True,False,False,19.333046
5,281,32,25895,3,5411,False,NaT,False,1,202.173.84.210,...,True,True,False,False,False,True,False,False,False,4.785622
6,300,30,0,4,2245,False,NaT,True,1,178.95.218.82,...,True,False,False,True,False,True,False,False,False,0.0
7,303,25,36223,5,2409,False,NaT,True,1,202.173.163.25,...,False,False,False,True,False,True,False,False,False,15.03653
8,330,35,42432,2,793,False,NaT,True,1,114.135.28.171,...,False,False,False,True,False,False,True,False,False,53.508197
9,353,46,0,3,8430,False,NaT,True,1,191.195.123.68,...,False,True,False,False,False,False,False,True,False,0.0


In [ ]:
## Transformation Task 4: Create a calculated field called Time_Since_Previous_Assistance
## Using Days? or something else?

df["Application_Date"] = pd.to_datetime(df["Application_Date"])
df["Previous_Assistance_Date"] = pd.to_datetime(df["Previous_Assistance_Date"])

df["Time_Since_Previous_Assistance"] = (
    df["Application_Date"] - df["Previous_Assistance_Date"]
).dt.days

df.head(10)

,Applicant_ID,Age,Income,Number_of_Dependents,Amount_Requested,Previous_Assistance_Received,Previous_Assistance_Date,Supporting_Doc_Verified,Application_Frequency_Last_Year,IP_Address,...,Device_Type_Desktop,Device_Type_Mobile,Device_Type_Tablet,Age_18_24,Age_25_34,Age_35_44,Age_45_54,Age_55_plus,Income_to_Amount_Requested,Time_Since_Previous_Assistance
0,217,65,28984,4,5872,False,NaT,False,1,156.133.45.45,...,False,True,False,False,False,False,False,True,4.935967,NaN
1,226,54,0,1,6631,False,NaT,False,1,245.13.80.245,...,False,False,True,False,False,False,True,False,0.0,NaN
2,240,26,64477,5,8612,False,NaT,True,1,213.103.170.95,...,False,True,False,False,True,False,False,False,7.486879,NaN
3,252,28,28576,4,2951,False,NaT,True,1,234.179.149.207,...,True,False,False,False,True,False,False,False,9.683497,NaN
4,266,43,44930,5,2324,False,NaT,False,1,66.109.96.227,...,False,True,False,False,False,True,False,False,19.333046,NaN
5,281,32,25895,3,5411,False,NaT,False,1,202.173.84.210,...,True,False,False,False,True,False,False,False,4.785622,NaN
6,300,30,0,4,2245,False,NaT,True,1,178.95.218.82,...,False,False,True,False,True,False,False,False,0.0,NaN
7,303,25,36223,5,2409,False,NaT,True,1,202.173.163.25,...,False,False,True,False,True,False,False,False,15.03653,NaN
8,330,35,42432,2,793,False,NaT,True,1,114.135.28.171,...,False,False,True,False,False,True,False,False,53.508197,NaN
9,353,46,0,3,8430,False,NaT,True,1,191.195.123.68,...,True,False,False,False,False,False,True,False,0.0,NaN


In [ ]:
## Transformation Task 5: Change True/False fields to 0s and 1s

bool_columns = df.select_dtypes(include="bool").columns
df[bool_columns] = df[bool_columns].astype(int)

df.head(10)


,Applicant_ID,Age,Income,Number_of_Dependents,Amount_Requested,Previous_Assistance_Received,Previous_Assistance_Date,Supporting_Doc_Verified,Application_Frequency_Last_Year,IP_Address,...,Device_Type_Desktop,Device_Type_Mobile,Device_Type_Tablet,Age_18_24,Age_25_34,Age_35_44,Age_45_54,Age_55_plus,Income_to_Amount_Requested,Time_Since_Previous_Assistance
0,217,65,28984,4,5872,0,NaT,0,1,156.133.45.45,...,0,1,0,0,0,0,0,1,4.935967,NaN
1,226,54,0,1,6631,0,NaT,0,1,245.13.80.245,...,0,0,1,0,0,0,1,0,0.0,NaN
2,240,26,64477,5,8612,0,NaT,1,1,213.103.170.95,...,0,1,0,0,1,0,0,0,7.486879,NaN
3,252,28,28576,4,2951,0,NaT,1,1,234.179.149.207,...,1,0,0,0,1,0,0,0,9.683497,NaN
4,266,43,44930,5,2324,0,NaT,0,1,66.109.96.227,...,0,1,0,0,0,1,0,0,19.333046,NaN
5,281,32,25895,3,5411,0,NaT,0,1,202.173.84.210,...,1,0,0,0,1,0,0,0,4.785622,NaN
6,300,30,0,4,2245,0,NaT,1,1,178.95.218.82,...,0,0,1,0,1,0,0,0,0.0,NaN
7,303,25,36223,5,2409,0,NaT,1,1,202.173.163.25,...,0,0,1,0,1,0,0,0,15.03653,NaN
8,330,35,42432,2,793,0,NaT,1,1,114.135.28.171,...,0,0,1,0,0,1,0,0,53.508197,NaN
9,353,46,0,3,8430,0,NaT,1,1,191.195.123.68,...,1,0,0,0,0,0,1,0,0.0,NaN


In [ ]:
## Remove Redundant Columns (Raw data before transformation)
df = df.drop(columns=[
    "Employment_Status",
    "Device_Type",
    "Age",
    "Application_Date",
    "Previous_Assistance_Date"
], errors="ignore")


In [ ]:
# Write the data back to BigQuery
table_id = f"{PROJECT_ID}.{DATASET_ID}.{TRAINING_TABLE}"

job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE"
)

job = client.load_table_from_dataframe(
    df,
    table_id,
    job_config=job_config
)

job.result()

LoadJob<project=qwiklabs-gcp-00-d269cced691e, location=US, id=94b995cc-1069-433c-9b46-dbe91b2b7b44>